In [1]:
import cudf
import seaborn as sns
import matplotlib.pyplot as plt


# gc.collect()

# Load CSV file into cuDF DataFrame
# df = cudf.read_csv('/rapids/notebooks/DATA/MASTER/merged.csv')

# here is the file list 
filelist = ['Wednesday-28-02-2018_TrafficForML_CICFlowMeter.parquet', 'Wednesday-21-02-2018_TrafficForML_CICFlowMeter.parquet', 'Wednesday-14-02-2018_TrafficForML_CICFlowMeter.parquet', 'Thursday-22-02-2018_TrafficForML_CICFlowMeter.parquet', 'Thursday-15-02-2018_TrafficForML_CICFlowMeter.parquet', 'Thursday-01-03-2018_TrafficForML_CICFlowMeter.parquet', 'Thursday-20-02-2018_TrafficForML_CICFlowMeter.parquet', 'Friday-23-02-2018_TrafficForML_CICFlowMeter.parquet', 'Friday-16-02-2018_TrafficForML_CICFlowMeter.parquet', 'Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet']

fname = filelist[9]
print(fname)

# files have been cleaned previously from CVS just checking no addtional header information has been added
# df = cudf.read_parquet('../DATA/PARQUET/'+fname)


fname_train = 'train_50'
fname_test = 'test_50'
df = cudf.read_parquet('../DATA/MASTER/'+fname_train+'.parquet')

# for training the timestamp data is of zero value 
df = df.drop('Timestamp', axis=1) 
display(df.shape)
df['Dst Port'] = df['Dst Port'].astype(str)
# lamda very useful mind only appear to work on integer - no concatenation of strings !!!!
df = df[df['Dst Port'].apply(lambda x: x.isdigit())]
display(df.tail())
display(df.shape)


Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet


(8068591, 79)

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
6550634,53,17,1001,1,1,38,66,38,38,38.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,Benign
16094478,445,6,487395,5,4,373,172,140,0,74.6,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
6423388,53,17,1791,1,1,42,102,42,42,42.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,Benign
15351219,49419,6,44,2,0,0,0,0,0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
14802022,53,17,311,1,1,38,54,38,38,38.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,Benign


(8068591, 79)

In [2]:
total_row = df.shape[0]

zero_counts = {}
# Iterate over columns
for column in df.columns:
    # Count number of zeros in the column and store in the dictionary
    zero_counts[column] = (df[column] == 0).sum()

# Convert the dictionary to a list of tuples and sort by count
sorted_counts = sorted(zero_counts.items(), key=lambda x: x[1], reverse=True)

# Print the sorted list
for i, (column, count) in enumerate(sorted_counts):
    print(f"Rank {i+1}: Column '{column}' has {count} zeros - {round((total_row-count)/total_row*100,2)}")
    
    

print("---")
print("---")
print("---")


nan_counts = {}
# Iterate over columns
for column in df.columns:
    # Count number of NaN values in the column and store in the dictionary
    nan_counts[column] = df[column].isnull().sum()

# Convert the dictionary to a list of tuples and sort by count
sorted_counts = sorted(nan_counts.items(), key=lambda x: x[1], reverse=True)

# Print the sorted list
for i, (column, count) in enumerate(sorted_counts):
    print(f"Rank {i+1}: Column '{column}' has {count} NaN values.")



Rank 1: Column 'Bwd PSH Flags' has 8068591 zeros - 0.0
Rank 2: Column 'Bwd URG Flags' has 8068591 zeros - 0.0
Rank 3: Column 'Fwd Byts/b Avg' has 8068591 zeros - 0.0
Rank 4: Column 'Fwd Pkts/b Avg' has 8068591 zeros - 0.0
Rank 5: Column 'Fwd Blk Rate Avg' has 8068591 zeros - 0.0
Rank 6: Column 'Bwd Byts/b Avg' has 8068591 zeros - 0.0
Rank 7: Column 'Bwd Pkts/b Avg' has 8068591 zeros - 0.0
Rank 8: Column 'Bwd Blk Rate Avg' has 8068591 zeros - 0.0
Rank 9: Column 'Fwd URG Flags' has 8067301 zeros - 0.02
Rank 10: Column 'CWE Flag Count' has 8067301 zeros - 0.02
Rank 11: Column 'FIN Flag Cnt' has 8033307 zeros - 0.44
Rank 12: Column 'Fwd PSH Flags' has 7729932 zeros - 4.2
Rank 13: Column 'SYN Flag Cnt' has 7729932 zeros - 4.2
Rank 14: Column 'URG Flag Cnt' has 7729466 zeros - 4.2
Rank 15: Column 'Active Std' has 7506477 zeros - 6.97
Rank 16: Column 'Idle Std' has 7425558 zeros - 7.97
Rank 17: Column 'Active Mean' has 7255779 zeros - 10.07
Rank 18: Column 'Active Max' has 7255779 zeros - 10.

In [3]:
df['Dst Port'] = df['Dst Port'].astype(int)
df['Protocol'] = df['Protocol'].astype(int)
df['Flow Duration'] = df['Flow Duration'].astype(int)
df['Tot Fwd Pkts'] = df['Tot Fwd Pkts'].astype(int)
df['Tot Bwd Pkts'] = df['Tot Bwd Pkts'].astype(int)
df['TotLen Fwd Pkts'] = df['TotLen Fwd Pkts'].astype(int)
df['TotLen Bwd Pkts'] = df['TotLen Bwd Pkts'].astype(int)
df['Fwd Pkt Len Max'] = df['Fwd Pkt Len Max'].astype(int)
df['Fwd Pkt Len Min'] = df['Fwd Pkt Len Min'].astype(int)
df['Fwd Pkt Len Mean'] = df['Fwd Pkt Len Mean'].astype(float)
df['Fwd Pkt Len Std'] = df['Fwd Pkt Len Std'].astype(float)
df['Bwd Pkt Len Max'] = df['Bwd Pkt Len Max'].astype(int)
df['Bwd Pkt Len Min'] = df['Bwd Pkt Len Min'].astype(int)
df['Bwd Pkt Len Mean'] = df['Bwd Pkt Len Mean'].astype(float)
df['Bwd Pkt Len Std'] = df['Bwd Pkt Len Std'].astype(float)
df['Flow Byts/s'] = df['Flow Byts/s'].astype(float)
df['Flow Pkts/s'] = df['Flow Pkts/s'].astype(float)
df['Flow IAT Mean'] = df['Flow IAT Mean'].astype(float)
df['Flow IAT Std'] = df['Flow IAT Std'].astype(float)
df['Flow IAT Max'] = df['Flow IAT Max'].astype(int)
df['Flow IAT Min'] = df['Flow IAT Min'].astype(int)
df['Fwd IAT Tot'] = df['Fwd IAT Tot'].astype(int)
df['Fwd IAT Mean'] = df['Fwd IAT Mean'].astype(float)
df['Fwd IAT Std'] = df['Fwd IAT Std'].astype(float)
df['Fwd IAT Max'] = df['Fwd IAT Max'].astype(int)
df['Fwd IAT Min'] = df['Fwd IAT Min'].astype(int)
df['Bwd IAT Tot'] = df['Bwd IAT Tot'].astype(int)
df['Bwd IAT Mean'] = df['Bwd IAT Mean'].astype(float)
df['Bwd IAT Std'] = df['Bwd IAT Std'].astype(float)
df['Bwd IAT Max'] = df['Bwd IAT Max'].astype(int)
df['Bwd IAT Min'] = df['Bwd IAT Min'].astype(int)
df['Fwd PSH Flags'] = df['Fwd PSH Flags'].astype(int)
df['Bwd PSH Flags'] = df['Bwd PSH Flags'].astype(int)
df['Fwd URG Flags'] = df['Fwd URG Flags'].astype(int)
df['Bwd URG Flags'] = df['Bwd URG Flags'].astype(int)
df['Fwd Header Len'] = df['Fwd Header Len'].astype(int)
df['Bwd Header Len'] = df['Bwd Header Len'].astype(int)
df['Fwd Pkts/s'] = df['Fwd Pkts/s'].astype(float)
df['Bwd Pkts/s'] = df['Bwd Pkts/s'].astype(float)
df['Pkt Len Min'] = df['Pkt Len Min'].astype(int)
df['Pkt Len Max'] = df['Pkt Len Max'].astype(int)
df['Pkt Len Mean'] = df['Pkt Len Mean'].astype(float)
df['Pkt Len Std'] = df['Pkt Len Std'].astype(float)
df['Pkt Len Var'] = df['Pkt Len Var'].astype(float)
df['FIN Flag Cnt'] = df['FIN Flag Cnt'].astype(int)
df['SYN Flag Cnt'] = df['SYN Flag Cnt'].astype(int)
df['RST Flag Cnt'] = df['RST Flag Cnt'].astype(int)
df['PSH Flag Cnt'] = df['PSH Flag Cnt'].astype(int)
df['ACK Flag Cnt'] = df['ACK Flag Cnt'].astype(int)
df['URG Flag Cnt'] = df['URG Flag Cnt'].astype(int)
df['CWE Flag Count'] = df['CWE Flag Count'].astype(int)
df['ECE Flag Cnt'] = df['ECE Flag Cnt'].astype(int)
df['Down/Up Ratio'] = df['Down/Up Ratio'].astype(int)
df['Pkt Size Avg'] = df['Pkt Size Avg'].astype(float)
df['Fwd Seg Size Avg'] = df['Fwd Seg Size Avg'].astype(float)
df['Bwd Seg Size Avg'] = df['Bwd Seg Size Avg'].astype(float)
df['Fwd Byts/b Avg'] = df['Fwd Byts/b Avg'].astype(int)
df['Fwd Pkts/b Avg'] = df['Fwd Pkts/b Avg'].astype(int)
df['Fwd Blk Rate Avg'] = df['Fwd Blk Rate Avg'].astype(int)
df['Bwd Byts/b Avg'] = df['Bwd Byts/b Avg'].astype(int)
df['Bwd Blk Rate Avg'] = df['Bwd Blk Rate Avg'].astype(int)
df['Subflow Fwd Pkts'] = df['Subflow Fwd Pkts'].astype(int)
df['Subflow Fwd Byts'] = df['Subflow Fwd Byts'].astype(int)
df['Subflow Bwd Pkts'] = df['Subflow Bwd Pkts'].astype(int)
df['Subflow Bwd Byts'] = df['Subflow Bwd Byts'].astype(int)
df['Init Fwd Win Byts'] = df['Init Fwd Win Byts'].astype(int)
df['Init Bwd Win Byts'] = df['Init Bwd Win Byts'].astype(int)
df['Fwd Act Data Pkts'] = df['Fwd Act Data Pkts'].astype(int)
df['Fwd Seg Size Min'] = df['Fwd Seg Size Min'].astype(int)
df['Active Mean'] = df['Active Mean'].astype(float)
df['Active Std'] = df['Active Std'].astype(float)
df['Active Max'] = df['Active Max'].astype(int)
df['Active Min'] = df['Active Min'].astype(int)
df['Idle Mean'] = df['Idle Mean'].astype(float)
df['Idle Std'] = df['Idle Std'].astype(float)
df['Idle Max'] = df['Idle Max'].astype(int)
df['Idle Min'] = df['Idle Min'].astype(int)

In [4]:
# df.iloc[0] not dask task used to save the parquest file and index number not in order!!!

unique_labels = df['Label'].unique()#
print(unique_labels)


label_frequencies = df['Label'].value_counts(normalize=True)
# Convert to percentage
label_percentages = label_frequencies * 100
# Convert the series to dataframe
label_percentages_df = label_percentages.reset_index()
# Rename the columns
label_percentages_df.columns = ['Label', 'Percentage']
# Print the DataFrame
print(label_percentages_df)

0               SSH-Bruteforce
1                       Benign
2             DoS attacks-Hulk
3                          Bot
4       DDoS attacks-LOIC-HTTP
5               FTP-BruteForce
6             DDOS attack-HOIC
7                Infilteration
8     DoS attacks-SlowHTTPTest
9        DoS attacks-GoldenEye
10       DoS attacks-Slowloris
11            Brute Force -Web
12        DDOS attack-LOIC-UDP
13            Brute Force -XSS
14               SQL Injection
Name: Label, dtype: object
                       Label  Percentage
0                     Benign   82.997528
1           DDOS attack-HOIC    4.250371
2     DDoS attacks-LOIC-HTTP    3.567575
3           DoS attacks-Hulk    2.859644
4                        Bot    1.772317
5             FTP-BruteForce    1.196628
6             SSH-Bruteforce    1.163388
7              Infilteration    0.988958
8   DoS attacks-SlowHTTPTest    0.863261
9      DoS attacks-GoldenEye    0.255435
10     DoS attacks-Slowloris    0.068736
11      DDOS att

In [5]:
dx = df[df['Label'] == 'Bot']
print(dx['Dst Port'].unique())

0        8080
1       51396
2       51553
3       50882
4       51040
        ...  
1156    51152
1157    51116
1158    51970
1159    50719
1160    50888
Name: Dst Port, Length: 1161, dtype: int64


In [6]:
from cudf import DataFrame, Series
from cuml.preprocessing import LabelEncoder
import gc

le = LabelEncoder()
encoded = le.fit_transform(df.Label)

# df = df.iloc[:10,:]
df = df.assign(encoded=encoded)
# df = df.drop(df.columns[0], axis=1)
LUT = df[['Label', 'encoded']]
LUT = LUT.drop_duplicates()


display(df.head())
# because of memory issues only using 9, for initial runs taken from papers 
X = df[['Fwd IAT Max', 'TotLen Fwd Pkts', 'Fwd IAT Std', 'Fwd Pkts/s', 'Fwd Header Len', 'Fwd Pkt Len Mean', 'Fwd Seg Size Min', 'Bwd IAT Tot', 'Fwd Pkt Len Max']]
y = df["encoded"]



# round Fwd Pkts to an integer
df['Fwd Pkts/s'] = df['Fwd Pkts/s'].round().astype('int')

# X = X.reset_index()
# # X = X.iloc[:, 1:10] 
# X = X.drop(X[0], axis=1)


# y = df["encoded"]
# y = y.reset_index()
# y = y.iloc[:, 1:2] 
# y = y['encoded']
# print(y.head())

# del df
# gc.collect()

# # Convert to DMatrix
# dtrain = xgb.DMatrix(data=X, label=y)

# # Specify parameters


# num_round = 10  # or any other number

# # Train
# bst = xgb.train(param, dtrain, num_round)

# # Make predictions
# preds_proba = bst.predict(dtrain)

# # For each sample, get indices of 5 classes with highest probabilities
# top_5_classes = (-preds_proba).argsort(axis=1)[:,:5]

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,encoded
5823274,22,6,6,1,1,0,0,0,0,0.0,...,0.0,0.0,0,0,0.0,0.0,0,0,SSH-Bruteforce,14
592703,11822,6,86065520,2,0,0,0,0,0,0.0,...,0.0,0.0,0,0,86100000.0,0.0,86100000,86100000,Benign,0
1372878,80,6,19459,2,0,0,0,0,0,0.0,...,0.0,0.0,0,0,0.0,0.0,0,0,DoS attacks-Hulk,8
4615014,37655,6,88049319,2,0,0,0,0,0,0.0,...,0.0,0.0,0,0,88049319.0,0.0,88049319,88049319,Benign,0
311923,8080,6,505,2,0,0,0,0,0,0.0,...,0.0,0.0,0,0,0.0,0.0,0,0,Bot,1


In [6]:
from xgboost import XGBClassifier
import time 
param = {'objective': 'multi:softprob', 'num_class': 9}  # where N is the number of classes

start_time = time.time()

classifier = XGBClassifier(tree_method='gpu_hist')
model = classifier.fit(X, y)
# predictions = model.predict(X_test)
# classification = classification_report(y_test, predictions)

end_time = time.time()
diff_time = end_time - start_time

print(f"Time: {diff_time:.4f} seconds")



In [10]:
from cuml.metrics import roc_auc_score
from cuml.metrics.accuracy import accuracy_score
import time

df = cudf.read_parquet('../DATA/MASTER/'+fname_test+'.parquet')

df = df.drop('Timestamp', axis=1)

df['Dst Port'] = df['Dst Port'].astype(int)
df['Protocol'] = df['Protocol'].astype(int)
df['Flow Duration'] = df['Flow Duration'].astype(int)
df['Tot Fwd Pkts'] = df['Tot Fwd Pkts'].astype(int)
df['Tot Bwd Pkts'] = df['Tot Bwd Pkts'].astype(int)
df['TotLen Fwd Pkts'] = df['TotLen Fwd Pkts'].astype(int)
df['TotLen Bwd Pkts'] = df['TotLen Bwd Pkts'].astype(int)
df['Fwd Pkt Len Max'] = df['Fwd Pkt Len Max'].astype(int)
df['Fwd Pkt Len Min'] = df['Fwd Pkt Len Min'].astype(int)
df['Fwd Pkt Len Mean'] = df['Fwd Pkt Len Mean'].astype(float)
df['Fwd Pkt Len Std'] = df['Fwd Pkt Len Std'].astype(float)
df['Bwd Pkt Len Max'] = df['Bwd Pkt Len Max'].astype(int)
df['Bwd Pkt Len Min'] = df['Bwd Pkt Len Min'].astype(int)
df['Bwd Pkt Len Mean'] = df['Bwd Pkt Len Mean'].astype(float)
df['Bwd Pkt Len Std'] = df['Bwd Pkt Len Std'].astype(float)
df['Flow Byts/s'] = df['Flow Byts/s'].astype(float)
df['Flow Pkts/s'] = df['Flow Pkts/s'].astype(float)
df['Flow IAT Mean'] = df['Flow IAT Mean'].astype(float)
df['Flow IAT Std'] = df['Flow IAT Std'].astype(float)
df['Flow IAT Max'] = df['Flow IAT Max'].astype(int)
df['Flow IAT Min'] = df['Flow IAT Min'].astype(int)
df['Fwd IAT Tot'] = df['Fwd IAT Tot'].astype(int)
df['Fwd IAT Mean'] = df['Fwd IAT Mean'].astype(float)
df['Fwd IAT Std'] = df['Fwd IAT Std'].astype(float)
df['Fwd IAT Max'] = df['Fwd IAT Max'].astype(int)
df['Fwd IAT Min'] = df['Fwd IAT Min'].astype(int)
df['Bwd IAT Tot'] = df['Bwd IAT Tot'].astype(int)
df['Bwd IAT Mean'] = df['Bwd IAT Mean'].astype(float)
df['Bwd IAT Std'] = df['Bwd IAT Std'].astype(float)
df['Bwd IAT Max'] = df['Bwd IAT Max'].astype(int)
df['Bwd IAT Min'] = df['Bwd IAT Min'].astype(int)
df['Fwd PSH Flags'] = df['Fwd PSH Flags'].astype(int)
df['Bwd PSH Flags'] = df['Bwd PSH Flags'].astype(int)
df['Fwd URG Flags'] = df['Fwd URG Flags'].astype(int)
# df['Bwd URG Flags'] = df['Bwd URG Flags'].astype(int)
df['Fwd Header Len'] = df['Fwd Header Len'].astype(int)
df['Bwd Header Len'] = df['Bwd Header Len'].astype(int)
df['Fwd Pkts/s'] = df['Fwd Pkts/s'].astype(float)
df['Bwd Pkts/s'] = df['Bwd Pkts/s'].astype(float)
df['Pkt Len Min'] = df['Pkt Len Min'].astype(int)
df['Pkt Len Max'] = df['Pkt Len Max'].astype(int)
df['Pkt Len Mean'] = df['Pkt Len Mean'].astype(float)
df['Pkt Len Std'] = df['Pkt Len Std'].astype(float)
df['Pkt Len Var'] = df['Pkt Len Var'].astype(float)
df['FIN Flag Cnt'] = df['FIN Flag Cnt'].astype(int)
df['SYN Flag Cnt'] = df['SYN Flag Cnt'].astype(int)
df['RST Flag Cnt'] = df['RST Flag Cnt'].astype(int)
df['PSH Flag Cnt'] = df['PSH Flag Cnt'].astype(int)
df['ACK Flag Cnt'] = df['ACK Flag Cnt'].astype(int)
df['URG Flag Cnt'] = df['URG Flag Cnt'].astype(int)
df['CWE Flag Count'] = df['CWE Flag Count'].astype(int)
df['ECE Flag Cnt'] = df['ECE Flag Cnt'].astype(int)
df['Down/Up Ratio'] = df['Down/Up Ratio'].astype(int)
df['Pkt Size Avg'] = df['Pkt Size Avg'].astype(float)
df['Fwd Seg Size Avg'] = df['Fwd Seg Size Avg'].astype(float)
df['Bwd Seg Size Avg'] = df['Bwd Seg Size Avg'].astype(float)
df['Fwd Byts/b Avg'] = df['Fwd Byts/b Avg'].astype(int)
df['Fwd Pkts/b Avg'] = df['Fwd Pkts/b Avg'].astype(int)
df['Fwd Blk Rate Avg'] = df['Fwd Blk Rate Avg'].astype(int)
df['Bwd Byts/b Avg'] = df['Bwd Byts/b Avg'].astype(int)
df['Bwd Blk Rate Avg'] = df['Bwd Blk Rate Avg'].astype(int)
df['Subflow Fwd Pkts'] = df['Subflow Fwd Pkts'].astype(int)
df['Subflow Fwd Byts'] = df['Subflow Fwd Byts'].astype(int)
df['Subflow Bwd Pkts'] = df['Subflow Bwd Pkts'].astype(int)
df['Subflow Bwd Byts'] = df['Subflow Bwd Byts'].astype(int)
df['Init Fwd Win Byts'] = df['Init Fwd Win Byts'].astype(int)
df['Init Bwd Win Byts'] = df['Init Bwd Win Byts'].astype(int)
df['Fwd Act Data Pkts'] = df['Fwd Act Data Pkts'].astype(int)
df['Fwd Seg Size Min'] = df['Fwd Seg Size Min'].astype(int)
df['Active Mean'] = df['Active Mean'].astype(float)
df['Active Std'] = df['Active Std'].astype(float)
df['Active Max'] = df['Active Max'].astype(int)
df['Active Min'] = df['Active Min'].astype(int)
df['Idle Mean'] = df['Idle Mean'].astype(float)
df['Idle Std'] = df['Idle Std'].astype(float)
df['Idle Max'] = df['Idle Max'].astype(int)
df['Idle Min'] = df['Idle Min'].astype(int)

df['Fwd Pkts/s'] = df['Fwd Pkts/s'].round().astype('int')

datanames = ['Fwd IAT Max', 'TotLen Fwd Pkts', 'Fwd IAT Std', 'Fwd Pkts/s', 'Fwd Header Len', 'Fwd Pkt Len Mean', 'Fwd Seg Size Min', 'Bwd IAT Tot', 'Fwd Pkt Len Max', 'Label']

df = df[datanames]

# for i in datanames:
#     print(df[i].dtype)

df['Fwd IAT Std'] = df['Fwd IAT Std'].round().astype(int)
df['Fwd Pkts/s'] = df['Fwd Pkts/s'].round().astype(int)
df['Fwd Pkt Len Mean'] = df['Fwd Pkt Len Mean'].round().astype(int)
    
# for i in datanames:
#     print(df[i].dtype)



    
X_test = df.drop(columns=["Label"])
y_test = df["Label"] 

for i, r in LUT.to_pandas().iterrows():
    # print(r[0])
    y_test = y_test.replace(r[0], str(r[1]))

y_test = y_test.astype(int)


start_time = time.time()

predictions = model.predict(X_test)

end_time = time.time()
diff_time = end_time - start_time

print(f"Time: {diff_time:.4f} seconds")

print("cuml XGboost Accuracy Score: ", accuracy_score(model.predict(X_test), y_test))




# # there appears to be NaN
# for col in features_test.columns:
#     features_test[col] = cudf.to_numeric(features_test[col], errors='coerce')
#     features_test[col] = features_test[col].fillna(features_test[col].median())

# del df
# gc.collect()

# target_pred = model.predict(features_test)

# accuracy = accuracy_score(target_test.to_array(), target_pred.to_array())
# print('Model accuracy: ', accuracy)


Time: 0.7566 seconds
cuml XGboost Accuracy Score:  0.9609179496765137


In [8]:
from sklearn.metrics import classification_report

classification = classification_report(y_test.to_pandas(), predictions)
print(classification)

              precision    recall  f1-score   support

           0       0.97      0.99      0.98   6693518
           1       0.90      0.98      0.94    143190
           2       0.97      0.37      0.53       295
           3       1.00      0.38      0.55       124
           4       0.91      0.97      0.94    343067
           5       0.80      0.85      0.83       887
           6       0.99      0.53      0.69    288338
           7       0.98      0.99      0.98     20898
           8       0.99      1.00      1.00    231179
           9       0.76      0.51      0.61     70237
          10       1.00      0.79      0.88      5444
          11       0.71      0.88      0.79     96803
          12       0.57      0.02      0.03     80844
          13       0.57      0.17      0.26        48
          14       1.00      1.00      1.00     93720

    accuracy                           0.96   8068592
   macro avg       0.87      0.70      0.73   8068592
weighted avg       0.96   